In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2

In [2]:
# load the face detection model
prototxtPath = './OpenCV_face_detector/architecture.prototxt'
weightsPath = './OpenCV_face_detector/weights.caffemodel'
faceDetector = cv2.dnn.readNet(prototxtPath, weightsPath)
# Now loading the face mask detector model we constructed earlier
maskDetector = load_model("./mask_detector.model")

WINDOWTITLE = "Real Time Mask Detection"

In [3]:
def SetColorAndLabel(improper, masked, unmasked):
    label=""
    color=(0,0,0)
    if masked == max(improper, masked, unmasked):
        label = "Masked"
        color = (0, 255, 0)
    elif unmasked == max(improper, masked, unmasked):
        label = "Unmasked"
        color = (0, 0, 255)
    else:
        label = "Improper"
        color = (0 , 255, 255)
    
    #Set display label to predicted class and confidence in predicted class
    pred = "{:.2f}".format(max(improper, masked, unmasked)*100) #convert to 2dp string
    label = f'{label} - conf:{pred}%'
    return(label,color)

In [4]:
def GetPredictionAndLocation(frame, faceDetector, maskDetector):
	faces = []
	locations = []
	predictions = []
	# Getting the dimensions of the frame to construct a blob 
	(height, width) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

	faceDetector.setInput(blob)
	detections = faceDetector.forward()

	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]
		# filtering out the week portions 
		if confidence > 0.5:
			box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
			(startX, startY, endX, endY) = box.astype("int")

			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(width - 1, endX), min(height - 1, endY))

			# Converting the color from BGR to RGB
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			# Preprocess extracted image to be compatible with mask detector model 
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			faces.append(face)
			locations.append((startX, startY, endX, endY))

	# Looking that is there any face to detect
	if len(faces) > 0:
		# Now the prediction is on batch base. Not on pictures one by one
		faces = np.array(faces, dtype="float32")
		predictions = maskDetector.predict(faces, batch_size=32)
	# Now returning 2 lists containing location and prediction of all the face detected
	return (locations , predictions)


In [7]:
#Start capturing using camera
print("Opening Camera")
vid = cv2.VideoCapture(0)
novid = cv2.imread('./OpenCV_face_detector/video-not-working.png')
while True:
	#grab frame from camera
	ret, frame = vid.read()
	if not ret:
		#if no frame captured, display error image
		cv2.imshow(WINDOWTITLE, novid)
	else:
		#Detect any faces in the frame. Predict mask class of detected face. Return location and prediction
		(locations , predictions) = GetPredictionAndLocation(frame, faceDetector, maskDetector)

		#For each face detected, get the location and prediction scores
		for (box, pred) in zip(locations , predictions):
			#get binding box dimensions
			(startX, startY, endX, endY) = box
			#get prediction score for each class
			(improper, masked, unmasked) = pred
			#set class with max prediction as final answer of model -> Adjust colors appropriately
			(label,color)=SetColorAndLabel(improper, masked, unmasked)

			
			#display class and confidence vertically above face
			cv2.putText(frame, label, (startX, startY - 5), cv2.FONT_HERSHEY_PLAIN, 0.8, color)
			#display rectangle at location of detected face
			cv2.rectangle(frame, (startX, startY), (endX, endY), color)
			cv2.imshow(WINDOWTITLE, frame)

	key = cv2.waitKey(1) & 0xFF

	#To Quit the Stream press (q)
	if key == ord("q"):
		break

#Release camera, close openCV windows
print("Closing Camera")
vid.release()
cv2.destroyAllWindows()

Opening Camera
Closing Camera


In [5]:
#Incase of crash, execute this block
vid.release()
cv2.destroyAllWindows()